# Generate RTX-KG2 Sample JSON to Parquet

In [1]:
import gzip
import json
import pathlib
import shutil
from typing import Any, Dict, Generator, List

import awkward as ak
import ijson
import pyarrow as pa
import requests
from genson import SchemaBuilder
from pyarrow import parquet
from rtx_kg2_functions import (
    count_items_under_top_level_name,
    find_top_level_names,
    parse_items_by_topmost_item_name,
    parse_metadata_by_object_name,
)

In [2]:
# set data to be used throughout notebook
chunk_size = 1
data_dir = "data"
parquet_dir = f"{data_dir}/"
source_data_url = "https://github.com/ncats/translator-lfs-artifacts/raw/main/files/kg2c_lite_2.8.4.json.gz"
target_extracted_sample_data = f"{data_dir}/{pathlib.Path(source_data_url).name.replace('.json.gz', '.sample.json')}"
parquet_dir = target_extracted_sample_data.replace(".json", ".dataset.parquet")
target_extracted_sample_data_schema_file = target_extracted_sample_data.replace(
    ".json", ".schema.json"
)
with open(target_extracted_sample_data_schema_file, "r") as file:
    # read the schema and and compress spacing within new schema string
    target_extracted_sample_data_schema = json.dumps(json.loads(file.read()))

In [3]:
pathlib.Path(parquet_dir).mkdir(exist_ok=True)

In [4]:
# show the top level object names for json file
top_level_names = list(find_top_level_names(target_extracted_sample_data))
print(top_level_names)

['nodes', 'edges', 'kg2_version', 'biolink_version']


In [5]:
# count the number of items under each top level name
top_level_name_item_counts = {
    top_level_name: count_items_under_top_level_name(
        target_extracted_sample_data, top_level_name
    )
    for top_level_name in top_level_names
}
print(top_level_name_item_counts)

{'nodes': 2, 'edges': 2, 'kg2_version': 0, 'biolink_version': 0}


In [6]:
# gather metadata names by inference from item count
metadata_top_level_names = [
    top_level_name
    for top_level_name, count in top_level_name_item_counts.items()
    if count == 0
]
metadata_top_level_names

['kg2_version', 'biolink_version']

In [7]:
# build a metadata dict
metadata_dict = dict(
    {
        metadata_top_level_name: parse_metadata_by_object_name(
            target_extracted_sample_data, metadata_top_level_name
        )
        for metadata_top_level_name in metadata_top_level_names
    },
    **{"source_data_json_schema": target_extracted_sample_data_schema},
)
metadata_dict

{'kg2_version': '2.8.4',
 'biolink_version': '3.1.2',
 'source_data_json_schema': '{"$schema": "http://json-schema.org/schema#", "$id": "https://github.com/CU-DBMI/rtx-kg2-gateway/src/notebooks/data/kg2c_lite_2.8.4.schema.json", "description": "Inferred JSON Schema from subset of RTX-KG2 JSON data for documenting data provenance.", "type": "object", "properties": {"nodes": {"type": "array", "items": {"type": "object", "properties": {"id": {"type": "string"}, "name": {"type": "string"}, "all_categories": {"type": "array", "items": {"type": "string"}}, "category": {"type": "string"}}, "required": ["all_categories", "category", "id", "name"]}}, "edges": {"type": "array", "items": {"type": "object", "properties": {"qualified_object_aspect": {"type": "string"}, "predicate": {"type": "string"}, "domain_range_exclusion": {"type": "string"}, "object": {"type": "string"}, "qualified_object_direction": {"type": "string"}, "id": {"type": "integer"}, "primary_knowledge_source": {"type": "string"},

In [8]:
# build a sample of data using limited number of items

# specify a map for from to specification
# to move these to first two cols of related table
edges_from_to_map = {"from": "subject", "to": "object"}

sample_items_dict = {}
for top_level_name in [
    name for name in top_level_names if name not in metadata_top_level_names
]:
    dataset_path = f"{parquet_dir}/{top_level_name}"
    pathlib.Path(dataset_path).mkdir(exist_ok=True)
    items = parse_items_by_topmost_item_name(
        target_extracted_sample_data, top_level_name, chunk_size, 0
    )
    for idx, value in enumerate(items):
        if top_level_name == "nodes":
            parquet.write_table(
                table=pa.Table.from_pylist(list(value)).replace_schema_metadata(
                    metadata_dict
                ),
                where=f"{dataset_path}/{top_level_name}.{idx}.parquet",
            )
        elif top_level_name == "edges":
            table = pa.Table.from_pylist(list(value))
            table = table.select(
                [edges_from_to_map["from"], edges_from_to_map["to"]]
                + [
                    name
                    for name in table.schema.names
                    if name not in edges_from_to_map.values()
                ]
            )

            parquet.write_table(
                table=table.replace_schema_metadata(metadata_dict),
                where=f"{dataset_path}/{top_level_name}.{idx}.parquet",
            )